In [1]:
# DEVICES = "CPU" # Moguce vrijednosti: "CPU", "GPU:0", "GPU:1", "GPUS"
import os
import logging
# os.environ['TF_CUDNN_DETERMINISTIC']='1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

print("Using only CPU")
os.environ["CUDA_VISIBLE_DEVICES"] = '-1'

logging.getLogger('tensorflow').setLevel(logging.FATAL)

# if DEVICES == 'CPU':
    # os.environ["CUDA_VISIBLE_DEVICES"] = '-1'
# elif DEVICES == 'GPU:0':
    # os.environ["CUDA_VISIBLE_DEVICES"] = '0'
# elif DEVICES == 'GPU:1':
    # os.environ["CUDA_VISIBLE_DEVICES"] = '1'


import tensorflow as tf

from dataset.dataset_loader import NerProcessor, FewNERDProcessor, create_tf_dataset_for_client, split_to_tf_datasets, batch_features
from utils.fl_utils import *
from models.model import build_BertNer, MaskedSparseCategoricalCrossentropy
from tokenization import FullTokenizer


from tqdm.notebook import tqdm
import json

import numpy as np

import nest_asyncio
nest_asyncio.apply()

import tensorflow_federated as tff

# if DEVICES == 'CPU':
    # cl_tf_devices = tf.config.list_logical_devices('CPU')
# elif DEVICES == 'GPUS':
    # cl_tf_devices = tf.config.list_logical_devices('GPU')
# else:
    # cl_tf_devices = tf.config.list_logical_devices('GPU')[:1]
# tff.backends.native.set_local_execution_context(
    # server_tf_device=tf.config.list_logical_devices('CPU')[0],
    # client_tf_devices=cl_tf_devices)


# Pretrained models
TINY = 'uncased_L-2_H-128_A-2'
TINY_12_128='uncased_L-12_H-128_A-2'
TINY_8_128='uncased_L-8_H-128_A-2'
TINY_6_128 ='uncased_L-6_H-128_A-2'
TINY_4_128='uncased_L-4_H-128_A-2'
TINY_2_512 ='uncased_L-2_H-512_A-8'
TINY_2_256='uncased_L-2_H-256_A-4'
MINI = 'uncased_L-4_H-256_A-4'
SMALL = 'uncased_L-4_H-512_A-8'
MEDIUM = 'uncased_L-8_H-512_A-8'
BASE = 'uncased_L-12_H-768_A-12'


MODEL = os.path.join("models", TINY_2_256)
SEQ_LEN = 128
BATCH_SIZE = 32
PRETRAINED = False


# processor = NerProcessor('dataset/conll')
# datas="conll"
processor = FewNERDProcessor('dataset/few_nerd')
datas="few_nerd"
tokenizer = FullTokenizer(os.path.join(MODEL, "vocab.txt"), True)
train_features = processor.get_train_as_features(SEQ_LEN, tokenizer)
eval_features = processor.get_test_as_features(SEQ_LEN, tokenizer)

# Name of file
modelSize=""
pretr=""
# change dataset name in datas

modelSize = {
  'models/uncased_L-2_H-128_A-2': lambda: "tiny",
  'models/uncased_L-12_H-128_A-2': lambda: "tiny_12_128",
  'models/uncased_L-8_H-128_A-2': lambda: "tiny_8_128",
  'models/uncased_L-6_H-128_A-2': lambda: "tiny_6_128",
  'models/uncased_L-2_H-512_A-8': lambda: "tiny_2_512",
  'models/uncased_L-4_H-128_A-2': lambda: "tiny_4_128",
  'models/uncased_L-2_H-256_A-4': lambda: "tiny_2_256",
  'models/uncased_L-4_H-256_A-4': lambda: "mini",
  'models/uncased_L-4_H-512_A-8': lambda: "small",
  'models/uncased_L-8_H-512_A-8': lambda: "medium",
  'models/uncased_L-12_H-768_A-12': lambda: "base"
}[MODEL]()

if PRETRAINED == True:
    pretr="pretrained"
elif PRETRAINED == False:
    pretr="nontrained"


# Wrap a Keras model for use with TFF.
def model_fn(m_name, num_labels, seq_len, input_spec):
    model = build_BertNer(m_name, num_labels, seq_len)
    return tff.learning.from_keras_model(
        model,
        input_spec=input_spec,
        loss=MaskedSparseCategoricalCrossentropy()) # reduction=tf.keras.losses.Reduction.NONE))

def eval_model(model, eval_data, do_print=True):
    return evaluate(model, eval_data, 
                    processor.get_label_map(), 
                    processor.token_ind('O'), 
                    processor.token_ind('[SEP]'),
                    processor.token_ind('[PAD]'), 
                    do_print=do_print)
    
# eval_features = eval_features[:5_000] # Samo prvih par tisuca za testiranje, maknuti za konačne ekperimente
eval_data_batched = batch_features(eval_features, processor.get_labels(), SEQ_LEN, tokenizer, batch_size=64)


Using only CPU


In [ ]:
num_clients, num_train_clients = 100, 10
assert num_clients >= num_train_clients

dataset_list = split_to_tf_datasets(train_features, num_clients, batch_size=BATCH_SIZE)
trainer = tff.learning.build_federated_averaging_process(
    model_fn=lambda: model_fn(MODEL, processor.label_len(), SEQ_LEN, dataset_list[0].element_spec),
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(5e-3),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adam(5e-3),
    use_experimental_simulation_loop=True
)

state = trainer.initialize()
if PRETRAINED:
    state = state_from_checkpoint(state, build_BertNer(MODEL, processor.label_len(), SEQ_LEN), MODEL)

res_list=[]
examples = 0
for rnd_ind in range(1, 501):
    train_data = list(np.random.choice(dataset_list, num_train_clients, replace=False))
    state, metrics = trainer.next(state, train_data)
    print("Round", rnd_ind, "Loss:", metrics['train']['loss'], "Examples:", metrics['stat']['num_examples'])
    examples = metrics['stat']['num_examples']
    
    # Ne treba svaku rundu gledati tocnost, moze svakih 10 (jedna epoha kumulativno)
    if rnd_ind % num_train_clients == 0:
        state_model = state_to_model(state, build_BertNer(MODEL, processor.label_len(), SEQ_LEN))
        res = eval_model(state_model, eval_data_batched, do_print=True)
        res['Round'] = rnd_ind
        res['Examples'] = examples
        res_list.append(res)

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

with open("log/results-"+modelSize+"-"+pretr+"-"+datas+".json", "w") as outfile:
    json.dump({'results': res_list, 'model': MODEL, 'seq_len': SEQ_LEN, 
               'pretrained': PRETRAINED, 'batch_size': BATCH_SIZE}, outfile, indent=None, cls=NpEncoder)

Round 1 Loss: 11.005543 Examples: 13120
Round 2 Loss: 3.5901573 Examples: 13120
Round 3 Loss: 8.532608 Examples: 13120
Round 4 Loss: 3.6534314 Examples: 13120
Round 5 Loss: 4.2872014 Examples: 13120
Round 6 Loss: 4.288045 Examples: 13120
Round 7 Loss: 2.6922534 Examples: 13120
Round 8 Loss: 2.046996 Examples: 13120
Round 9 Loss: 1.6617575 Examples: 13120
Round 10 Loss: 1.6115149 Examples: 13120


Evaluating:   0%|          | 0/589 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         ART     0.0000    0.0000    0.0000      4064
         BUI     0.0000    0.0000    0.0000      5007
         EVE     0.0000    0.0000    0.0000      4104
         LOC     0.0000    0.0000    0.0000     27227
        MISC     0.0000    0.0000    0.0000      9558
         ORG     0.0000    0.0000    0.0000     19077
         PER     0.0000    0.0000    0.0000     21527
         PRO     0.0000    0.0000    0.0000      6230

   micro avg     0.0000    0.0000    0.0000     96794
   macro avg     0.0000    0.0000    0.0000     96794
weighted avg     0.0000    0.0000    0.0000     96794

Round 11 Loss: 1.5534637 Examples: 13120
Round 12 Loss: 1.6543257 Examples: 13120
Round 13 Loss: 1.3055265 Examples: 13120
Round 14 Loss: 1.3306916 Examples: 13120
Round 15 Loss: 1.4259505 Examples: 13120
Round 16 Loss: 1.517979 Examples: 13120
Round 17 Loss: 1.5002304 Examples: 13120
Round 18 Loss: 1.3287886 Examples: 13120
Round 19 Loss: 1.35344

Evaluating:   0%|          | 0/589 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         ART     0.0000    0.0000    0.0000      4064
         BUI     0.0000    0.0000    0.0000      5007
         EVE     0.0000    0.0000    0.0000      4104
         LOC     0.0000    0.0000    0.0000     27227
        MISC     0.0000    0.0000    0.0000      9558
         ORG     0.0000    0.0000    0.0000     19077
         PER     0.0000    0.0000    0.0000     21527
         PRO     0.0000    0.0000    0.0000      6230

   micro avg     0.0000    0.0000    0.0000     96794
   macro avg     0.0000    0.0000    0.0000     96794
weighted avg     0.0000    0.0000    0.0000     96794

Round 21 Loss: 1.3306322 Examples: 13120
Round 22 Loss: 1.3346608 Examples: 13120
Round 23 Loss: 1.2921785 Examples: 13120
Round 24 Loss: 1.2885451 Examples: 13120
Round 25 Loss: 1.3013893 Examples: 13120
Round 26 Loss: 1.2495424 Examples: 13120
Round 27 Loss: 1.2443753 Examples: 13120
Round 28 Loss: 1.2370862 Examples: 13120
Round 29 Loss: 1.2331

Evaluating:   0%|          | 0/589 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         ART     0.0000    0.0000    0.0000      4064
         BUI     0.0000    0.0000    0.0000      5007
         EVE     0.0000    0.0000    0.0000      4104
         LOC     0.0000    0.0000    0.0000     27227
        MISC     0.0000    0.0000    0.0000      9558
         ORG     0.0000    0.0000    0.0000     19077
         PER     0.0000    0.0000    0.0000     21527
         PRO     0.0000    0.0000    0.0000      6230

   micro avg     0.0000    0.0000    0.0000     96794
   macro avg     0.0000    0.0000    0.0000     96794
weighted avg     0.0000    0.0000    0.0000     96794

Round 31 Loss: 1.2611247 Examples: 13120
Round 32 Loss: 1.2549887 Examples: 13120
Round 33 Loss: 1.2329527 Examples: 13120
Round 34 Loss: 1.2305228 Examples: 13120
Round 35 Loss: 1.2246093 Examples: 13120
Round 36 Loss: 1.221482 Examples: 13120
Round 37 Loss: 1.2168652 Examples: 13120
Round 38 Loss: 1.2225299 Examples: 13120
Round 39 Loss: 1.22149

Evaluating:   0%|          | 0/589 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         ART     0.0000    0.0000    0.0000      4064
         BUI     0.0000    0.0000    0.0000      5007
         EVE     0.0000    0.0000    0.0000      4104
         LOC     0.0000    0.0000    0.0000     27227
        MISC     0.0000    0.0000    0.0000      9558
         ORG     0.0000    0.0000    0.0000     19077
         PER     0.0000    0.0000    0.0000     21527
         PRO     0.0000    0.0000    0.0000      6230

   micro avg     0.0000    0.0000    0.0000     96794
   macro avg     0.0000    0.0000    0.0000     96794
weighted avg     0.0000    0.0000    0.0000     96794

Round 41 Loss: 1.2130344 Examples: 13120
Round 42 Loss: 1.2189611 Examples: 13120
Round 43 Loss: 1.2128942 Examples: 13120
Round 44 Loss: 1.2090988 Examples: 13120
